In [1]:
import cv2 as cv
import numpy as np
import pickle
import glob

In [2]:
with open('/lab/vislab/DATA/ActualFullTraffickCam/hotel2img_dict.pkl','rb') as f:
    hotel2img = pickle.load(f)

In [3]:
with open('/lab/vislab/DATA/ActualFullTraffickCam/hotelimageinfo.pkl','rb') as f:
    hotelimageinfo = pickle.load(f)

<ipython-input-3-5d8d724c9084>:2: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  hotelimageinfo = pickle.load(f)


In [4]:
import matplotlib.pyplot as plt

In [5]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/lab/vislab/models/r2d2')

import os, pdb
from PIL import Image
import numpy as np
import torch

from tools import common
from tools.dataloader import norm_RGB
from nets.patchnet import *

In [6]:
def load_network(model_fn): 
    checkpoint = torch.load(model_fn)
    print("\n>> Creating net = " + checkpoint['net']) 
    net = eval(checkpoint['net'])
    nb_of_weights = common.model_size(net)
    print(f" ( Model size: {nb_of_weights/1000:.0f}K parameters )")

    # initialization
    weights = checkpoint['state_dict']
    net.load_state_dict({k.replace('module.',''):v for k,v in weights.items()})
    return net.eval()

In [7]:
class NonMaxSuppression (torch.nn.Module):
    def __init__(self, rel_thr=0.7, rep_thr=0.7):
        nn.Module.__init__(self)
        self.max_filter = torch.nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.rel_thr = rel_thr
        self.rep_thr = rep_thr
    
    def forward(self, reliability, repeatability, **kw):
        assert len(reliability) == len(repeatability) == 1
        reliability, repeatability = reliability[0], repeatability[0]

        # local maxima
        maxima = (repeatability == self.max_filter(repeatability))

        # remove low peaks
        maxima *= (repeatability >= self.rep_thr)
        maxima *= (reliability   >= self.rel_thr)

        return maxima.nonzero().t()[2:4]

In [8]:
def extract_multiscale( net, img, detector, scale_f=2**0.25, 
                        min_scale=0.0, max_scale=1, 
                        min_size=256, max_size=1024, 
                        verbose=False):
    old_bm = torch.backends.cudnn.benchmark 
    torch.backends.cudnn.benchmark = False # speedup
    
    # extract keypoints at multiple scales
    B, three, H, W = img.shape
    assert B == 1 and three == 3, "should be a batch with a single RGB image"
    
    assert max_scale <= 1
    s = 1.0 # current scale factor
    
    X,Y,S,C,Q,D = [],[],[],[],[],[]
    while  s+0.001 >= max(min_scale, min_size / max(H,W)):
        if s-0.001 <= min(max_scale, max_size / max(H,W)):
            nh, nw = img.shape[2:]
            if verbose: print(f"extracting at scale x{s:.02f} = {nw:4d}x{nh:3d}")
            # extract descriptors
            with torch.no_grad():
                res = net(imgs=[img])
                
            # get output and reliability map
            descriptors = res['descriptors'][0]
            reliability = res['reliability'][0]
            repeatability = res['repeatability'][0]

            # normalize the reliability for nms
            # extract maxima and descs
            y,x = detector(**res) # nms
            c = reliability[0,0,y,x]
            q = repeatability[0,0,y,x]
            d = descriptors[0,:,y,x].t()
            n = d.shape[0]

            # accumulate multiple scales
            X.append(x.float() * W/nw)
            Y.append(y.float() * H/nh)
            S.append((32/s) * torch.ones(n, dtype=torch.float32, device=d.device))
            C.append(c)
            Q.append(q)
            D.append(d)
            
            del res
            torch.cuda.empty_cache()
            
        s /= scale_f

        # down-scale the image for next iteration
        nh, nw = round(H*s), round(W*s)
        img = F.interpolate(img, (nh,nw), mode='bilinear', align_corners=False)

    # restore value
    torch.backends.cudnn.benchmark = old_bm

    Y = torch.cat(Y)
    X = torch.cat(X)
    S = torch.cat(S) # scale
    scores = torch.cat(C) * torch.cat(Q) # scores = reliability * repeatability
    XYS = torch.stack([X,Y,S], dim=-1)
    D = torch.cat(D)
    return XYS, D, scores

In [9]:
# load the network...
net = load_network('/lab/vislab/models/r2d2/models/r2d2_WASF_N8_big.pt')
net = net.cuda()
net.eval()

# create the non-maxima detector
detector = NonMaxSuppression(rel_thr = 0.7, rep_thr = 0.7)



>> Creating net = Quad_L2Net_ConfCFS(mchan=6)
 ( Model size: 1041K parameters )


In [10]:
import os

feat_dir = '/lab/vislab/DATA/tcam_local_features'

completed = [c.split('.')[0].split('/')[-1] for c in glob.glob(os.path.join(feat_dir,'*.pkl'))]

In [11]:
list(hotel2img.keys())[0] in completed

True

In [12]:
num_hotels = len(list(hotel2img.keys()))
num_completed = 0
for hotel in list(hotel2img.keys()):
    if hotel not in completed:
        hotel_dict = {}
        for im_obj in hotel2img[hotel]:
            with torch.no_grad():
                im_id = int(im_obj['img'].split('/')[-1].split('_')[0])
                img_path = im_obj['img']

                img = Image.open(img_path).convert('RGB')
                W, H = img.size
                img = norm_RGB(img)[None] 
                img = img.cuda()

                # extract keypoints/descriptors for a single image
                try:
                    xys, desc, scores = extract_multiscale(net, img, detector,
                        scale_f   = 2**0.25, 
    #                     min_scale = 0.3, 
    #                     max_scale = 1,
    #                     min_size  = 0, 
    #                     max_size  = 9999, 
                        verbose = False)

                    xys = xys.cpu().numpy()
                    desc = desc.cpu().numpy()
                    scores = scores.cpu().numpy()
                    idxs = scores.argsort()[None:]

                    hotel_dict[im_id] = {'xys':xys, 'desc':desc, 'scores':scores, 'idxs':idxs}
                except:
                    continue
                    
                del img
                torch.cuda.empty_cache()

        outfile = os.path.join(feat_dir,str(hotel)+'.pkl')
        with open(outfile, 'wb') as handle:
            pickle.dump(hotel_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

        del hotel_dict
        torch.cuda.empty_cache()

    if num_completed % 5 == 0:
        print(num_completed, num_hotels, num_completed / num_hotels)

    num_completed += 1

0 361127 0.0
5 361127 1.384554464218959e-05
10 361127 2.769108928437918e-05
15 361127 4.153663392656877e-05
20 361127 5.538217856875836e-05
25 361127 6.922772321094795e-05
30 361127 8.307326785313753e-05
35 361127 9.691881249532713e-05
40 361127 0.00011076435713751672
45 361127 0.0001246099017797063
50 361127 0.0001384554464218959
55 361127 0.0001523009910640855
60 361127 0.00016614653570627507
65 361127 0.00017999208034846466
70 361127 0.00019383762499065426
75 361127 0.00020768316963284386
80 361127 0.00022152871427503343
85 361127 0.00023537425891722303
90 361127 0.0002492198035594126
95 361127 0.0002630653482016022
100 361127 0.0002769108928437918
105 361127 0.0002907564374859814
110 361127 0.000304601982128171
115 361127 0.0003184475267703606
120 361127 0.00033229307141255013
125 361127 0.00034613861605473973
130 361127 0.00035998416069692933
135 361127 0.00037382970533911893
140 361127 0.0003876752499813085
145 361127 0.0004015207946234981
150 361127 0.0004153663392656877
155 361

1260 361127 0.0034890772498317765
1265 361127 0.003502922794473966
1270 361127 0.0035167683391161557
1275 361127 0.0035306138837583453
1280 361127 0.003544459428400535
1285 361127 0.0035583049730427245
1290 361127 0.003572150517684914
1295 361127 0.0035859960623271037
1300 361127 0.0035998416069692933
1305 361127 0.003613687151611483
1310 361127 0.0036275326962536725
1315 361127 0.003641378240895862
1320 361127 0.0036552237855380517
1325 361127 0.0036690693301802413
1330 361127 0.003682914874822431
1335 361127 0.0036967604194646205
1340 361127 0.00371060596410681
1345 361127 0.0037244515087489997
1350 361127 0.0037382970533911893
1355 361127 0.003752142598033379
1360 361127 0.0037659881426755685
1365 361127 0.003779833687317758
1370 361127 0.0037936792319599477
1375 361127 0.0038075247766021373
1380 361127 0.003821370321244327
1385 361127 0.0038352158658865165
1390 361127 0.003849061410528706
1395 361127 0.0038629069551708957
1400 361127 0.0038767524998130853
1405 361127 0.003890598044

2520 361127 0.006978154499663553
2525 361127 0.006992000044305743
2530 361127 0.007005845588947932
2535 361127 0.007019691133590122
2540 361127 0.007033536678232311
2545 361127 0.0070473822228745014
2550 361127 0.007061227767516691
2555 361127 0.007075073312158881
2560 361127 0.00708891885680107
2565 361127 0.00710276440144326
2570 361127 0.007116609946085449
2575 361127 0.007130455490727639
2580 361127 0.007144301035369828
2585 361127 0.007158146580012018
2590 361127 0.007171992124654207
2595 361127 0.007185837669296397
2600 361127 0.007199683213938587
2605 361127 0.007213528758580777
2610 361127 0.007227374303222966
2615 361127 0.007241219847865156
2620 361127 0.007255065392507345
2625 361127 0.007268910937149535
2630 361127 0.007282756481791724
2635 361127 0.007296602026433914
2640 361127 0.007310447571076103
2645 361127 0.007324293115718293
2650 361127 0.007338138660360483
2655 361127 0.007351984205002673
2660 361127 0.007365829749644862
2665 361127 0.007379675294287052
2670 361127

3780 361127 0.010467231749495329
3785 361127 0.01048107729413752
3790 361127 0.01049492283877971
3795 361127 0.010508768383421898
3800 361127 0.010522613928064087
3805 361127 0.010536459472706278
3810 361127 0.010550305017348468
3815 361127 0.010564150561990657
3820 361127 0.010577996106632846
3825 361127 0.010591841651275037
3830 361127 0.010605687195917226
3835 361127 0.010619532740559415
3840 361127 0.010633378285201604
3845 361127 0.010647223829843795
3850 361127 0.010661069374485984
3855 361127 0.010674914919128174
3860 361127 0.010688760463770363
3865 361127 0.010702606008412554
3870 361127 0.010716451553054743
3875 361127 0.010730297097696932
3880 361127 0.010744142642339121
3885 361127 0.010757988186981312
3890 361127 0.010771833731623501
3895 361127 0.01078567927626569
3900 361127 0.01079952482090788
3905 361127 0.01081337036555007
3910 361127 0.01082721591019226
3915 361127 0.010841061454834449
3920 361127 0.010854906999476638
3925 361127 0.010868752544118829
3930 361127 0.01

5045 361127 0.013970154543969297
5050 361127 0.013984000088611486
5055 361127 0.013997845633253675
5060 361127 0.014011691177895864
5065 361127 0.014025536722538055
5070 361127 0.014039382267180244
5075 361127 0.014053227811822434
5080 361127 0.014067073356464623
5085 361127 0.014080918901106812
5090 361127 0.014094764445749003
5095 361127 0.014108609990391192
5100 361127 0.014122455535033381
5105 361127 0.01413630107967557
5110 361127 0.014150146624317761
5115 361127 0.01416399216895995
5120 361127 0.01417783771360214
5125 361127 0.014191683258244329
5130 361127 0.01420552880288652
5135 361127 0.014219374347528709
5140 361127 0.014233219892170898
5145 361127 0.014247065436813087
5150 361127 0.014260910981455278
5155 361127 0.014274756526097467
5160 361127 0.014288602070739656
5165 361127 0.014302447615381846
5170 361127 0.014316293160024036
5175 361127 0.014330138704666226
5180 361127 0.014343984249308415
5185 361127 0.014357829793950604
5190 361127 0.014371675338592795
5195 361127 0.

...

351700 361127 0.9738956101316157
351705 361127 0.973909455676258
351710 361127 0.9739233012209001
351715 361127 0.9739371467655423
351720 361127 0.9739509923101846
351725 361127 0.9739648378548267
351730 361127 0.9739786833994689
351735 361127 0.973992528944111
351740 361127 0.9740063744887533
351745 361127 0.9740202200333955
351750 361127 0.9740340655780376
351755 361127 0.9740479111226799
351760 361127 0.974061756667322
351765 361127 0.9740756022119642
351770 361127 0.9740894477566064
351775 361127 0.9741032933012486
351780 361127 0.9741171388458908
351785 361127 0.9741309843905329
351790 361127 0.9741448299351751
351795 361127 0.9741586754798174
351800 361127 0.9741725210244595
351805 361127 0.9741863665691017
351810 361127 0.974200212113744
351815 361127 0.9742140576583861
351820 361127 0.9742279032030283
351825 361127 0.9742417487476704
351830 361127 0.9742555942923127
351835 361127 0.9742694398369549
351840 361127 0.974283285381597
351845 361127 0.9742971309262393
351850 361127 0

353545 361127 0.9790046161045837
353550 361127 0.9790184616492259
353555 361127 0.9790323071938681
353560 361127 0.9790461527385103
353565 361127 0.9790599982831525
353570 361127 0.9790738438277946
353575 361127 0.9790876893724368
353580 361127 0.9791015349170791
353585 361127 0.9791153804617212
353590 361127 0.9791292260063634
353595 361127 0.9791430715510056
353600 361127 0.9791569170956478
353605 361127 0.97917076264029
353610 361127 0.9791846081849321
353615 361127 0.9791984537295744
353620 361127 0.9792122992742166
353625 361127 0.9792261448188587
353630 361127 0.979239990363501
353635 361127 0.9792538359081431
353640 361127 0.9792676814527853
353645 361127 0.9792815269974275
353650 361127 0.9792953725420697
353655 361127 0.9793092180867119
353660 361127 0.979323063631354
353665 361127 0.9793369091759963
353670 361127 0.9793507547206385
353675 361127 0.9793646002652806
353680 361127 0.9793784458099228
353685 361127 0.9793922913545651
353690 361127 0.9794061368992072
353695 361127 

355655 361127 0.9848474359435877
355660 361127 0.9848612814882299
355665 361127 0.984875127032872
355670 361127 0.9848889725775143
355675 361127 0.9849028181221565
355680 361127 0.9849166636667986
355685 361127 0.9849305092114409
355690 361127 0.984944354756083
355695 361127 0.9849582003007252
355700 361127 0.9849720458453675
355705 361127 0.9849858913900096
355710 361127 0.9849997369346518
355715 361127 0.9850135824792939
355720 361127 0.9850274280239362
355725 361127 0.9850412735685784
355730 361127 0.9850551191132205
355735 361127 0.9850689646578628
355740 361127 0.985082810202505
355745 361127 0.9850966557471471
355750 361127 0.9851105012917893
355755 361127 0.9851243468364315
355760 361127 0.9851381923810737
355765 361127 0.9851520379257159
355770 361127 0.985165883470358
355775 361127 0.9851797290150003
355780 361127 0.9851935745596424
355785 361127 0.9852074201042846
355790 361127 0.9852212656489269
355795 361127 0.985235111193569
355800 361127 0.9852489567382112
355805 361127 0

358210 361127 0.9919225092557465
358215 361127 0.9919363548003888
358220 361127 0.991950200345031
358225 361127 0.9919640458896731
358230 361127 0.9919778914343154
358235 361127 0.9919917369789576
358240 361127 0.9920055825235997
358245 361127 0.992019428068242
358250 361127 0.9920332736128841
358255 361127 0.9920471191575263
358260 361127 0.9920609647021685
358265 361127 0.9920748102468107
358270 361127 0.9920886557914529
358275 361127 0.992102501336095
358280 361127 0.9921163468807372
358285 361127 0.9921301924253795
358290 361127 0.9921440379700216
358295 361127 0.9921578835146638
358300 361127 0.992171729059306
358305 361127 0.9921855746039482
358310 361127 0.9921994201485904
358315 361127 0.9922132656932325
358320 361127 0.9922271112378748
358325 361127 0.992240956782517
358330 361127 0.9922548023271591
358335 361127 0.9922686478718014
358340 361127 0.9922824934164435
358345 361127 0.9922963389610857
358350 361127 0.9923101845057279
358355 361127 0.9923240300503701
358360 361127 0

359900 361127 0.9966023033448067
359905 361127 0.9966161488894488
359910 361127 0.9966299944340911
359915 361127 0.9966438399787333
359920 361127 0.9966576855233754
359925 361127 0.9966715310680176
359930 361127 0.9966853766126598
359935 361127 0.996699222157302
359940 361127 0.9967130677019442
359945 361127 0.9967269132465864
359950 361127 0.9967407587912286
359955 361127 0.9967546043358707
359960 361127 0.9967684498805129
359965 361127 0.9967822954251552
359970 361127 0.9967961409697973
359975 361127 0.9968099865144395
359980 361127 0.9968238320590818
359985 361127 0.9968376776037239
359990 361127 0.9968515231483661
359995 361127 0.9968653686930082
360000 361127 0.9968792142376505
360005 361127 0.9968930597822927
360010 361127 0.9969069053269348
360015 361127 0.996920750871577
360020 361127 0.9969345964162192
360025 361127 0.9969484419608614
360030 361127 0.9969622875055036
360035 361127 0.9969761330501458
360040 361127 0.996989978594788
360045 361127 0.9970038241394302
360050 361127